In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, udf, regexp_replace, lit, from_unixtime
from pyspark.sql.types import ArrayType, StringType, StructType, StructField, IntegerType, StringType, MapType
from pyspark.sql.functions import split
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, MapType, StringType
from pyspark.sql.functions import explode


from pyspark.sql.functions import split, explode, regexp_extract, col, collect_list, udf
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, DoubleType
from pyspark.ml.linalg import VectorUDT, Vectors
import numpy as np

In [9]:
#Initialize Spark session
spark = SparkSession.builder.appName("BehaviorsProcessing").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/12/07 16:07:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/07 16:07:16 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/12/07 16:07:16 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 58962)
Traceback (most recent call last):
  File "/usr/lib/python3.9/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.9/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.9/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.9/socketserver.py", line 720, in __init__
    self.handle()
  File "/usr/local/lib/python3.9/dist-packages/pyspark/accumulators.py", line 281, in handle
    poll(accum_updates)
  File "/usr/local/lib/python3.9/dist-packages/pyspark/accumulators.py", line 253, in poll
    if func():
  File "/usr/local/lib/python3.9/dist-packages/pyspark/accumulators.py", line 257, in accum_updates
    num_updates = read_int(self.rfile)
  File "/usr/lo

In [10]:
# Define the schema
behaviors_schema = StructType([
    StructField("ImpressionID", StringType(), True),
    StructField("UserID", StringType(), True),
    StructField("Time", StringType(), True),
    StructField("History", StringType(), True),
    StructField("Impressions", StringType(), True)
])

# Load the behaviors.tsv file
behaviors_df = spark.read.csv(
    "data/mind/MINDsmall_train/behaviors.tsv",
    sep="\t",
    schema=behaviors_schema,
    header=False
)

# Display the schema and a sample row
behaviors_df.printSchema()
behaviors_df.show(1, truncate=False)

root
 |-- ImpressionID: string (nullable = true)
 |-- UserID: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- History: string (nullable = true)
 |-- Impressions: string (nullable = true)

+------------+------+---------------------+--------------------------------------------------------------+-----------------+
|ImpressionID|UserID|Time                 |History                                                       |Impressions      |
+------------+------+---------------------+--------------------------------------------------------------+-----------------+
|1           |U13740|11/11/2019 9:05:58 AM|N55189 N42782 N34694 N45794 N18445 N63302 N10414 N19347 N31801|N55689-1 N35729-0|
+------------+------+---------------------+--------------------------------------------------------------+-----------------+
only showing top 1 row



In [11]:
# Split History into an array
behaviors_df = behaviors_df.withColumn("HistoryList", split(col("History"), " "))
behaviors_df = behaviors_df.drop("History")  # Drop original History column if not needed

# Verify the transformation
behaviors_df.select("ImpressionID", "UserID", "HistoryList").show(1, truncate=False)

+------------+------+------------------------------------------------------------------------+
|ImpressionID|UserID|HistoryList                                                             |
+------------+------+------------------------------------------------------------------------+
|1           |U13740|[N55189, N42782, N34694, N45794, N18445, N63302, N10414, N19347, N31801]|
+------------+------+------------------------------------------------------------------------+
only showing top 1 row



In [12]:
# Split Impressions into an array
behaviors_df = behaviors_df.withColumn("ImpressionsList", split(col("Impressions"), " "))
behaviors_df = behaviors_df.drop("Impressions")  # Drop original Impressions column if not needed

# Verify the transformation
behaviors_df.select("ImpressionID", "ImpressionsList").show(1, truncate=False)

+------------+--------------------+
|ImpressionID|ImpressionsList     |
+------------+--------------------+
|1           |[N55689-1, N35729-0]|
+------------+--------------------+
only showing top 1 row



In [13]:
# Explode ImpressionsList
impressions_exploded = behaviors_df.select(
    "ImpressionID",
    "UserID",
    "Time",
    "HistoryList",
    explode("ImpressionsList").alias("ImpressionItem")
)

# Extract CandidateNewsID and ClickLabel using regex
impressions_exploded = impressions_exploded \
    .withColumn("CandidateNewsID", regexp_extract(col("ImpressionItem"), r"^(N\d+)-\d+$", 1)) \
    .withColumn("ClickLabel", regexp_extract(col("ImpressionItem"), r"^N\d+-(\d+)$", 1).cast("integer")) \
    .drop("ImpressionItem")

# Verify the transformation
impressions_exploded.select("ImpressionID", "UserID", "CandidateNewsID", "ClickLabel").show(5, truncate=False)

+------------+------+---------------+----------+
|ImpressionID|UserID|CandidateNewsID|ClickLabel|
+------------+------+---------------+----------+
|1           |U13740|N55689         |1         |
|1           |U13740|N35729         |0         |
|2           |U91836|N20678         |0         |
|2           |U91836|N39317         |0         |
|2           |U91836|N58114         |0         |
+------------+------+---------------+----------+
only showing top 5 rows



In [14]:
news_tfidf_path = "news_tfidf.parquet"
news_features_df = spark.read.parquet(news_tfidf_path)

In [15]:
news_features_df.show()

[Stage 5:>                                                          (0 + 1) / 1]

+------+--------------------+
|NewsID|       TFIDFFeatures|
+------+--------------------+
| N5727|(10000,[1,9,87,90...|
|N25908|(10000,[165,280,4...|
| N2490|(10000,[1,218,262...|
|  N192|(10000,[71,2006,2...|
| N1298|(10000,[3,26,42,4...|
|N57313|(10000,[36,61,109...|
|N36185|(10000,[6,21,74,8...|
|N33743|(10000,[2,13,16,3...|
|N58255|(10000,[0,10,14,3...|
|N44291|(10000,[0,4,15,84...|
|N38233|(10000,[5,7,11,69...|
| N1970|(10000,[4,26,30,3...|
|N41692|(10000,[4,12,31,1...|
|N31209|(10000,[4,34,59,1...|
|N60452|(10000,[154,159,2...|
|N22043|(10000,[36,119,25...|
|N30368|(10000,[45,815,12...|
| N4233|(10000,[11,90,121...|
|N51387|(10000,[111,865,1...|
|N22126|(10000,[4,18,34,3...|
+------+--------------------+
only showing top 20 rows



In [16]:
# Join impressions with news_features_df on CandidateNewsID
impressions_with_features = impressions_exploded.join(
    news_features_df,
    impressions_exploded.CandidateNewsID == news_features_df.NewsID,
    how="left"
).drop(news_features_df.NewsID)  # Drop duplicate NewsID column if present


In [17]:
# Verify the join
impressions_with_features.select("ImpressionID", "UserID", "CandidateNewsID", "ClickLabel", "TFIDFFeatures").show(1, truncate=False)

+------------+------+---------------+----------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ImpressionID|UserID|CandidateNewsID|ClickLabel|TFIDFFeatures                                                                                                                                                                                                                                                                                                                                                                                                                                  |
+------------+------+---------------+-

In [18]:
# Filter records where ClickLabel == 1
clicked_news_df = impressions_with_features.filter(col("ClickLabel") == 1)

# Verify the filtered DataFrame
clicked_news_df.show(5, truncate=True)


+------------+------+--------------------+--------------------+---------------+----------+--------------------+
|ImpressionID|UserID|                Time|         HistoryList|CandidateNewsID|ClickLabel|       TFIDFFeatures|
+------------+------+--------------------+--------------------+---------------+----------+--------------------+
|           1|U13740|11/11/2019 9:05:5...|[N55189, N42782, ...|         N55689|         1|(10000,[8,30,51,9...|
|           2|U91836|11/12/2019 6:11:3...|[N31739, N6072, N...|         N17059|         1|(10000,[25,38,108...|
|           3|U73700|11/14/2019 7:01:4...|[N10732, N25792, ...|         N23814|         1|(10000,[43,100,14...|
|           4|U34670|11/11/2019 5:28:0...|[N45729, N2203, N...|         N49685|         1|(10000,[128,170,2...|
|           5| U8125|11/12/2019 4:11:2...|[N10078, N56514, ...|          N8400|         1|(10000,[1,63,79,1...|
+------------+------+--------------------+--------------------+---------------+----------+--------------

In [19]:
# Group by UserID and collect all TFIDFFeatures into a list
user_clicks_df = clicked_news_df.groupBy("UserID") \
    .agg(collect_list("TFIDFFeatures").alias("ClickedTFIDFList"))

# Verify the aggregation
user_clicks_df.show(1, truncate=True)

[Stage 11:>                                                         (0 + 8) / 8]

+------+--------------------+
|UserID|    ClickedTFIDFList|
+------+--------------------+
|U10022|[(10000,[37,152,1...|
+------+--------------------+
only showing top 1 row



In [20]:
from pyspark.sql.functions import udf
from pyspark.ml.linalg import VectorUDT, DenseVector

# Define the UDF to compute average vector
def average_vector(vectors):
    if not vectors:
        return Vectors.sparse(10000, [])  # Adjust size if different
    sum_array = np.sum([v.toArray() for v in vectors], axis=0)
    mean_array = sum_array / len(vectors)
    return Vectors.dense(mean_array)

# Register the UDF
average_vector_udf = udf(average_vector, VectorUDT())

# Apply the UDF to create UserProfile
user_profiles_df = user_clicks_df.withColumn("UserProfile", average_vector_udf(col("ClickedTFIDFList"))) \
    .select("UserID", "UserProfile")

# Verify the user profiles
user_profiles_df.show(1, truncate=True)

[Stage 17:>                                                         (0 + 1) / 1]

+------+--------------------+
|UserID|         UserProfile|
+------+--------------------+
|U10022|[0.53004950030122...|
+------+--------------------+
only showing top 1 row



In [21]:
from pyspark.sql.functions import explode, regexp_extract

# Explode the 'ImpressionsList' into individual 'ImpressionItem' rows
impressions_exploded = behaviors_df.select(
    "ImpressionID",
    "UserID",
    "HistoryList",
    explode("ImpressionsList").alias("ImpressionItem")
)

# Extract 'CandidateNewsID' and 'ClickLabel' using regex
impressions_exploded = impressions_exploded \
    .withColumn("CandidateNewsID", regexp_extract(col("ImpressionItem"), r"^(N\d+)-\d+$", 1)) \
    .withColumn("ClickLabel", regexp_extract(col("ImpressionItem"), r"^N\d+-(\d+)$", 1).cast("integer")) \
    .drop("ImpressionItem")

# Verify the transformation
impressions_exploded.select("ImpressionID", "UserID", "CandidateNewsID", "ClickLabel").show(5, truncate=False)
behaviors_df.select("ImpressionID", "UserID", "HistoryList", "ImpressionsList").show(1, truncate=False)


+------------+------+---------------+----------+
|ImpressionID|UserID|CandidateNewsID|ClickLabel|
+------------+------+---------------+----------+
|1           |U13740|N55689         |1         |
|1           |U13740|N35729         |0         |
|2           |U91836|N20678         |0         |
|2           |U91836|N39317         |0         |
|2           |U91836|N58114         |0         |
+------------+------+---------------+----------+
only showing top 5 rows

+------------+------+------------------------------------------------------------------------+--------------------+
|ImpressionID|UserID|HistoryList                                                             |ImpressionsList     |
+------------+------+------------------------------------------------------------------------+--------------------+
|1           |U13740|[N55189, N42782, N34694, N45794, N18445, N63302, N10414, N19347, N31801]|[N55689-1, N35729-0]|
+------------+------+-------------------------------------------------

In [22]:
from pyspark.sql.functions import explode, regexp_extract

# Explode the 'ImpressionsList' into individual 'ImpressionItem' rows
impressions_exploded = behaviors_df.select(
    "ImpressionID",
    "UserID",
    "HistoryList",
    explode("ImpressionsList").alias("ImpressionItem")
)

# Extract 'CandidateNewsID' and 'ClickLabel' using regex
impressions_exploded = impressions_exploded \
    .withColumn("CandidateNewsID", regexp_extract(col("ImpressionItem"), r"^(N\d+)-\d+$", 1)) \
    .withColumn("ClickLabel", regexp_extract(col("ImpressionItem"), r"^N\d+-(\d+)$", 1).cast("integer")) \
    .drop("ImpressionItem")

# Verify the transformation
impressions_exploded.select("ImpressionID", "UserID", "CandidateNewsID", "ClickLabel").show(5, truncate=False)

+------------+------+---------------+----------+
|ImpressionID|UserID|CandidateNewsID|ClickLabel|
+------------+------+---------------+----------+
|1           |U13740|N55689         |1         |
|1           |U13740|N35729         |0         |
|2           |U91836|N20678         |0         |
|2           |U91836|N39317         |0         |
|2           |U91836|N58114         |0         |
+------------+------+---------------+----------+
only showing top 5 rows



In [23]:
impressions_with_features = impressions_exploded.join(
    news_features_df,
    impressions_exploded.CandidateNewsID == news_features_df.NewsID,
    how="left"
).drop(news_features_df.NewsID)  # Remove duplicate 'NewsID' column if present

# Verify the join
impressions_with_features.select("ImpressionID", "UserID", "CandidateNewsID", "ClickLabel", "TFIDFFeatures").show(5, truncate=True)

+------------+------+---------------+----------+--------------------+
|ImpressionID|UserID|CandidateNewsID|ClickLabel|       TFIDFFeatures|
+------------+------+---------------+----------+--------------------+
|           1|U13740|         N55689|         1|(10000,[8,30,51,9...|
|           1|U13740|         N35729|         0|(10000,[1,56,90,9...|
|           2|U91836|         N20678|         0|(10000,[21,44,82,...|
|           2|U91836|         N39317|         0|(10000,[8,10,16,2...|
|           2|U91836|         N58114|         0|                null|
+------------+------+---------------+----------+--------------------+
only showing top 5 rows



In [24]:
from pyspark.sql.functions import explode

# Explode the 'HistoryList' to get individual 'PastNewsID's
user_history_exploded = behaviors_df.select(
    "UserID",
    explode("HistoryList").alias("PastNewsID")
)

# Verify the exploded history
user_history_exploded.show(5, truncate=False)

+------+----------+
|UserID|PastNewsID|
+------+----------+
|U13740|N55189    |
|U13740|N42782    |
|U13740|N34694    |
|U13740|N45794    |
|U13740|N18445    |
+------+----------+
only showing top 5 rows



In [25]:
# Join with news_features_df to get TFIDFFeatures for past clicked news
user_history_with_features = user_history_exploded.join(
    news_features_df,
    user_history_exploded.PastNewsID == news_features_df.NewsID,
    how="left"
).drop(news_features_df.NewsID)

# Verify the join
user_history_with_features.select("UserID", "PastNewsID", "TFIDFFeatures").show(5, truncate=True)

+------+----------+--------------------+
|UserID|PastNewsID|       TFIDFFeatures|
+------+----------+--------------------+
|U13740|    N55189|(10000,[1,5,11,22...|
|U13740|    N42782|(10000,[13,18,37,...|
|U13740|    N34694|(10000,[154,2065,...|
|U13740|    N45794|(10000,[4,34,52,1...|
|U13740|    N18445|(10000,[34,102,15...|
+------+----------+--------------------+
only showing top 5 rows



In [26]:
from pyspark.sql.functions import collect_list, udf
from pyspark.ml.linalg import VectorUDT, Vectors
import numpy as np

# Define a UDF to compute the average vector
def average_vectors(vectors):
    if not vectors:
        return Vectors.sparse(10000, [])  # Adjust size if different
    sum_array = np.sum([v.toArray() for v in vectors], axis=0)
    mean_array = sum_array / len(vectors)
    return Vectors.dense(mean_array)

average_vectors_udf = udf(average_vectors, VectorUDT())

# Group by UserID and collect all TFIDFFeatures into a list
user_profiles = user_history_with_features.groupBy("UserID") \
    .agg(collect_list("TFIDFFeatures").alias("ClickedTFIDFList")) \
    .withColumn("UserProfile", average_vectors_udf(col("ClickedTFIDFList"))) \
    .select("UserID", "UserProfile")

# Verify the user profiles
user_profiles.show(1, truncate=True)

[Stage 29:>                                                         (0 + 1) / 1]

24/12/07 16:08:24 WARN PythonUDFRunner: Detected deadlock while completing task 0.0 in stage 29 (TID 96): Attempting to kill Python Worker
+------+--------------------+
|UserID|         UserProfile|
+------+--------------------+
|U10022|[0.44170791691768...|
+------+--------------------+
only showing top 1 row

